You can download and run this notebook locally, or you can run it for free in a cloud environment using Colab or Sagemaker Studio Lab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kirbyju/TCIA_Notebooks/blob/main/CPTAC/CPTAC.ipynb)

[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github.com/kirbyju/TCIA_Notebooks/blob/main/CPTAC/CPTAC.ipynb)

# Accessing DICOM images and annotations from the CPTAC datasets hosted on TCIA

This notebook is focused on accessing the **"Clinical Proteomic Tumor Analysis Consortium"** collections hosted on [The Cancer Imaging Archive(TCIA)](https://cancerimagingarchive.net).  These datasets include radiology and histopathology images hosted on TCIA as well as proteomic, genomic and clinical data hosted in the Proteomic Data Commons and Genomic Data Commons.  The National Cancer Institute has also funded an activity to generate and publish annotations (3d segmentation labels and seed points) on TCIA to help jumpstart research on tumor detection, auto-segmentation methods and generating related imaging features which can be compared with the proteomic, genomic and clinical data.  

**Note:** Users of this notebook will likely find the [CPTAC python package](https://github.com/PayneLab/cptac) useful for working with the non-image data types.


# 1 Setup

The following installs and imports **[tcia_utils](https://pypi.org/project/tcia-utils/)**, which contains a variety of useful functions for accessing TCIA via Python and Juptyter Notebooks.  It also ensures the necessary imports are performed.  Finally, there are a few conditional steps that will execute if you're using Google Colab to adjust log settings.

In [ ]:
import sys

# install tcia utils
!{sys.executable} -m pip install --upgrade -q tcia_utils

In [ ]:
import requests
import pandas as pd
from tcia_utils import nbia

# set logging level to INFO in Google Colab (not necessary in Jupyter)
if 'google.colab' in sys.modules:
  import logging

  for handler in logging.root.handlers[:]:
      logging.root.removeHandler(handler)

  # Set handler with level = info
  logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',
                      level=logging.INFO)

  print("Google Colab Logging set to INFO")

# 2 Learn about the datasets

The images, annotations (tumor segmentation and seed point labels), and other related data are described in detail at the following links.  These pages are publicly visible without logging in:

1. [CPTAC-UCEC](https://doi.org/10.7937/89M3-KQ43): Corpus Endometrial Carcinoma
2. [CPTAC-PDA](https://doi.org/10.7937/BW9V-BX61): Pancreatic Ductal Adenocarcinoma
3. [CPTAC-CCRCC](https://doi.org/10.7937/SKQ4-QX48): Clear Cell Renal Carcinoma
4. [CPTAC-HNSCC](https://doi.org/10.7937/PFEC-T641): Head and Neck Squamous Cell Carcinoma **(restricted access - requires extra steps below)**

The data in the CPTAC-HNSCC collection contains images that could potentially be used to reconstruct a human face. To safeguard the privacy of participants, users must sign and submit a [TCIA Restricted License Agreement](https://wiki.cancerimagingarchive.net/download/attachments/4556915/TCIA%20Restricted%20License%2020220519.pdf?version=1&modificationDate=1652964581655&api=v2) to help@cancerimagingarchive.net before accessing the image data used to create the tumor annotations.  

After taking a look at these collections, select the one you'd like to explore through the rest of this notebook by setting the collection variable below.

In [ ]:
collection = "CPTAC-PDA"

# 3 Downloading images and annotations with the NBIA Data Retriever

TCIA uses software called NBIA to manage its DICOM data.  One way to download TCIA data is to install the NBIA Data Retriever and use the predefined manifest files that are found on the summary pages mentioned in section 2.  

This tool provides a number of useful features such as auto-retry if there are any problems, saving data in an organized hierarchy on your hard drive (Collection > Patient > Study > Series > Images), and providing a CSV file containing key DICOM metadata about the images you've downloaded.  

There are versions available for Windows, Mac and Linux.  If you're working from a system with a GUI you can follow the [instructions](https://wiki.cancerimagingarchive.net/display/NBIA/Downloading+TCIA+Images) to install Data Retriever on your computer.  There is also a [Linux command-line version of the NBIA Data Retriever](https://wiki.cancerimagingarchive.net/x/2QKPBQ) which is demonstrated in [this notebook](https://github.com/kirbyju/TCIA_Notebooks/blob/main/TCIA_Linux_Data_Retriever_App.ipynb).

# 4 Accessing the REST APIs
The [NBIA REST APIs](https://wiki.cancerimagingarchive.net/x/ZoATBg) are another useful way for TCIA users to query metadata and download image data, which will be the focus of the rest of this notebook.  We'll rely heavily on [tcia_utils](https://pypi.org/project/tcia-utils/) to simplify accessing them.

If you have questions that are not covered in this notebook you can find many additional examples in the other notebooks at https://github.com/kirbyju/TCIA_Notebooks.

Before we get started, we need to set the api_url parameter to "restricted" and create an API access token for those of you accessing CPTAC-HNSCC.  If you selected a different collection, this will just set the api_url parameter to access the public APIs that don't require logging in.

In [ ]:
if collection == "CPTAC-HNSCC":
    api_url = "restricted"
    nbia.getToken()
else:
    api_url = ""

## 4.1 Explore the data with REST API queries

Let's start by looking at what body parts and modalities are contained in the collection.  For these datasets, RTSTRUCTs were used to record  the segmentations and seed points as well as to indicate scans where no tumor was found. By default, most functions from **tcia_utils** return results in JSON.

In [ ]:
# count patients for each modality
data = nbia.getModalityCounts(collection)
print(data)

However, you can also use **format = "df"** to return the results as a dataframe.

In [ ]:
# Count patients for each body part examined,
# return results as dataframe
df = nbia.getBodyPartCounts(collection, format = "df")

# rename headers and sort by PatientCount
df.rename(columns = {'criteria':'BodyPartExamined', 'count':'PatientCount'}, inplace = True)
df.PatientCount = df.PatientCount.astype(int)
display(df.sort_values(by='PatientCount', ascending=False, ignore_index = True))

Now let's run **nbia.getPatient()** and **nbia.getStudy()** to see what we can learn about the patient cohort from the DICOM metadata.  The patient information can include things like age, gender, and ethnicity. The study information includes additional information recorded on the date the patient was scanned such as the patient's age or how many days it has been since they were diagnosed.

In [ ]:
df = nbia.getPatient(collection, format = "df", api_url = api_url)

display(df)

Let's use **format = "csv"** this time to save a CSV file in addition to returning a dataframe.  Verify that **getPatientStudy.csv** has been saved to your file system before proceeding.

In [ ]:
# obtain study/visit details (e.g. anonymized study date, age at the time of visit)
df = nbia.getStudy(collection, format = "csv", api_url = api_url)
display(df)

We can also create a report with **nbia.getSeries()** that gives useful metadata about each scan in the dataset (e.g. series description, modality, scanner manufacturer & software version, number of images).

In [ ]:
# obtain scan/series metadata and save to variable for use in next example
data = nbia.getSeries(collection, api_url = api_url, format = "df")

display(data)

Finally, we can use the results from the getSeries() query to generate some summary statistics about the data in the collection.  Note that there are separate rows summarizing the contents of the original collection and the contents of the annotation dataset.

In [ ]:
# Calculate summary statistics for a given collection
nbia.reportDoiSummary(data, api_url = api_url, input_type = "df")

## 4.3 Downloading data with the REST API
Next we'll demonstrate using the API to download data.  This can be useful if you'd like to download results from API queries rather than using an existing manifest file.  It's also useful if you can't install the NBIA Data Retriever or want to integrate TCIA downloads into other pipelines/tools.  

Here we will focus on the following use cases:

1. Download and visualize a sample case
2. Download seed point labels
2. Download 3d segmentation labels
3. Download source images used to create seed points and segmentations
4. Download source images with negative finding assessments

To identify the subsets for our use cases, we'll leverage the **annotation metadata** spreadsheet the authors provided, which you can download manually from the **Annotation Summary** links in section 2 of the notebook or retrieve directly into a dataframe with the code below.

In [ ]:
metadata_urls = {
    "CPTAC-CCRCC": "https://www.cancerimagingarchive.net/wp-content/uploads/Metadata_Report_CPTAC-CCRCC_2023_07_14.csv",
    "CPTAC-PDA": "https://www.cancerimagingarchive.net/wp-content/uploads/Metadata_Report_CPTAC-PDA_2023_07_14.csv",
    "CPTAC-HNSCC": "https://www.cancerimagingarchive.net/wp-content/uploads/Metadata_Report_CPTAC-HNSCC_2023_07_14.csv",
    "CPTAC-UCEC": "https://www.cancerimagingarchive.net/wp-content/uploads/Metadata_Report_CPTAC-UCEC_2023_07_14.csv"
}

if collection in metadata_urls:
    spreadsheet_url = metadata_urls[collection]
    annotation_Metadata = pd.read_csv(spreadsheet_url)
    display(annotation_Metadata)
else:
    print("URL for collection not found.")

### Download and visualize a sample case
Here we'll walk through some steps to identify an example segmentation file, find the corresponding reference series and visualize them together in the notebook.

First, let's pull the series UID of the first segmentation from our annotation metadata.

In [ ]:
segSeries = annotation_Metadata.loc[annotation_Metadata['Annotation Type'] == 'Segmentation', 'SeriesInstanceUID'].iloc[0]

print(segSeries)

In this case we are fortunate to also have the corresponding reference Series UID in the spreadsheet so we can obtain that in a similar fashion.

In [ ]:
# Find the row where 'SeriesInstanceUID' is equal to segSeries
filtered_row = annotation_Metadata[annotation_Metadata['SeriesInstanceUID'] == segSeries]

# Extract the value from 'ReferencedSeriesInstanceUID' column in the filtered row
refSeries = filtered_row['ReferencedSeriesInstanceUID'].iloc[0]

print(refSeries)

Alternatively, if you ever have a situation where you don't have a spreadsheet like this and want to determine the Reference Series UID you can try using **nbia.getSegRefSeries()** to obtain it.

In [ ]:
refSeries = nbia.getSegRefSeries(segSeries)

print(refSeries)

Next let's download these two series.

In [ ]:
nbia.downloadSeries([refSeries, segSeries], input_type= "list", api_url = api_url)

Now we can look at the images and segmentation together with **viewSeriesAnnotation()**.  Note that this function is only meant to be a  quick and dirty way to preview the data.  There are more comprehensive solutions such as [3D Slicer](https://slicer.org/) or [itkWidgets](https://github.com/kirbyju/TCIA_Notebooks/blob/main/TCIA_RTStruct_SEG_Visualization_with_itkWidgets.ipynb) if you want analyze the data.

In [ ]:
nbia.viewSeriesAnnotation(seriesUid = refSeries, annotationUid = segSeries)

#### Download seed points
Since we're working with Series UIDs from a dataframe instead of JSON output from the API, we'll use the  **input_type = "list"** parameter in the remaining download steps.  The examples will download the first three series.  You can remove the **number = 3** parameter to download the entire dataset.  We'll also specify a **csv_filename** to save the related metadata to a file.

In [ ]:
# filter dataframe to only include seed point rows
seedPoints = annotation_Metadata[annotation_Metadata['Annotation Type'].str.contains('Seed point')]
#display(seedPoints)

# extract series UID column to list for downloading
series_data = seedPoints["SeriesInstanceUID"].tolist()

# download a sample set of three scans
# return metadata dataframe as dataframe
# save a CSV of the metadata
nbia.downloadSeries(series_data, number = 3, api_url = api_url, input_type = "list", csv_filename = collection + "-seedPoints")

#### Download 3D segmentations

In [ ]:
# filter dataframe to only include segmentations
segs = annotation_Metadata[annotation_Metadata['Annotation Type'].str.contains('Segmentation')]
#display(segs)

# extract series UID column to list for downloading
series_data = segs["SeriesInstanceUID"].tolist()

# download a sample set of three scans
# return metadata dataframe as dataframe
# save a CSV of the metadata
nbia.downloadSeries(series_data, number = 3, api_url = api_url, input_type = "list", csv_filename = collection + "-segs")

#### Download source images for seed points and segmentations

In [ ]:
# filter dataframe to only include seg and seed point rows (remove "no findings")
ref_Series = annotation_Metadata[(annotation_Metadata['Annotation Type'] == 'Seed point') |
                                 (annotation_Metadata['Annotation Type'] == 'Segmentation')]

# remove duplicate ReferencedSeriesUIDs
clean_refSeries = ref_Series.drop_duplicates(subset='ReferencedSeriesInstanceUID')
#display(clean_refSeries)

# extract series UID column to list for downloading
series_data = clean_refSeries["ReferencedSeriesInstanceUID"].tolist()

# download a sample set of three scans
# return metadata dataframe as dataframe
# save a CSV of the metadata
nbia.downloadSeries(series_data, number = 3, api_url = api_url, input_type = "list", csv_filename = collection + "-seg_seed_source_images")

The following code will download the scans with negative finding assessments.  These are cases where the authors of the dataset did not find anything that could be annotated.  Downloading these scans could be useful if you are training a tumor/metastases detection model.

In [ ]:
# filter dataframe to only include scans with "no findings"
ref_Series = annotation_Metadata[annotation_Metadata['Annotation Type'] == 'No findings']

# remove duplicate ReferencedSeriesUIDs
clean_refSeries = ref_Series.drop_duplicates(subset='ReferencedSeriesInstanceUID')
#display(clean_refSeries)

# extract series UID column to list for downloading
series_data = clean_refSeries["ReferencedSeriesInstanceUID"].tolist()

# download a sample set of three scans
# return metadata dataframe as dataframe
# save a CSV of the metadata
nbia.downloadSeries(series_data, number = 3, api_url = api_url, input_type = "list", csv_filename = collection + "-noFinding_source_images")

# Acknowledgements
[The Cancer Imaging Archive (TCIA)](https://www.cancerimagingarchive.net/) is a service which de-identifies and hosts a large publicly available archive of medical images of cancer.  TCIA is funded by the [Cancer Imaging Program (CIP)](https://imaging.cancer.gov/), a part of the United States [National Cancer Institute (NCI)](https://www.cancer.gov/), and is managed by the [Frederick National Laboratory for Cancer Research (FNLCR)](https://frederick.cancer.gov/).

This notebook was created by [Justin Kirby](https://www.linkedin.com/in/justinkirby82/).  If you leverage TCIA datasets in your work please be sure to comply with the [TCIA Data Usage Policy](https://wiki.cancerimagingarchive.net/x/c4hF) and include all relevant citations.

## TCIA Citation

Clark, K., Vendt, B., Smith, K., Freymann, J., Kirby, J., Koppel, P., Moore, S., Phillips, S., Maffitt, D., Pringle, M., Tarbox, L., & Prior, F. (2013). The Cancer Imaging Archive (TCIA): Maintaining and Operating a Public Information Repository. Journal of Digital Imaging, 26(6), 1045–1057. https://doi.org/10.1007/s10278-013-9622-7

## Dataset Citations

Citations for each dataset can be found on their summary pages that are listed in section 1.